In [1]:
from langchain_community.document_loaders import CSVLoader
from langchain_openai import ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter

csv_load = CSVLoader('netflix_titles_1000.csv', metadata_columns=['show_id','date_added','release_year'])
docs = csv_load.load()
splitter = RecursiveCharacterTextSplitter(separators=["\n",","," ",""],chunk_size=70,chunk_overlap=10)

chunks = splitter.split_documents(docs)

c:\Users\gaura\anaconda3\envs\langchain\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dotenv import load_dotenv,find_dotenv
import os
env_path = find_dotenv(filename='password.env',usecwd=True)
load_dotenv(env_path)
key = os.getenv('OPENAI_API_KEY')
#key

In [3]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model='text-embedding-3-small')
vector_store = Chroma.from_documents(documents=chunks, embedding = embedding_model)



In [4]:
retriever = vector_store.as_retriever(search_type='similarity',
                                      search_kwargs={'k':3})

In [5]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model = 'gpt-4o-mini', api_key=key )

prompt = ChatPromptTemplate.from_template("""You are an expert movie reccomendation engine.
                                          Based on the context, answer the question. The context is. 
                                          If the question is unrelated to movies say you do not know
                                          context: {context}
                                          question: {question}""")
chain = ({'context':retriever,'question':RunnablePassthrough()}
         | prompt
         | llm
         |StrOutputParser())

result = chain.invoke("""question: I love action movies. 
                      Recommend 2 action movies and in 
                      1 line explain the reason for the reccomendation""")

In [6]:
print(result)

1. **The Wrestler (2008)** - This film combines intense action with a compelling character study in the world of professional wrestling, highlighting the struggles and triumphs within the sport.

2. **Warrior (2011)** - A gripping drama that revolves around mixed martial arts, showcasing high-stakes action and deep emotional ties among the characters competing in the ring.
